In [30]:
# 재현성을 위해, 난수 생성하는 코드로 고정시켜 둠
import numpy as np
import csv
import time
np.random.seed(1234)
def randomize(): np.random.seed(time.time())

# 20200106
## 1장 

In [10]:
RND_MEAN = 0
RND_STD = 0.0030
LEARNING_RATE = 0.001

In [1]:
import pandas as pd
data = pd.read_csv("abalone.csv")

In [6]:
data

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [9]:
# 책 내부에 있는 코드 
def load_abalone_dataset():
    with open('./abalone.csv') as csvfile:
         csvreader = csv.reader(csvfile)
         next(csvreader, None)
         rows = []
         for row in csvreader:
            rows.append(row)
     
    global data, input_cnt, output_cnt # global 변수로 지정해줌
    
    # regression으로 숫자를 예측하기 위해 output은 1개
    # input이 10인 이유는 9+1
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt]) # 0으로 채워진 table을 만들고, 전처리에 쓸 자료 채울 예정 
    
    # one-hot encoding(문자로 되어 있는 categorical data를 이용) sex를 숫자로 바꿔주어야 함 
    for n,row in enumerate(rows):
        if row[0] == 'I' : data [n,0] =1
        if row[0] == 'M' : data [n,1] =1
        if row[0] == 'F' : data [n,2] =1   
        data[n, 3:] = row[1:] # 성별이 0,1,2열을 차지하고, 3부터는 원래의 데이터로 채움 

In [12]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [20]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) # 주로 전체 데이터 크기 / mini batch 크기 
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count): # epoch: 처음부터 끝까지 학습 시킨 횟수 
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1)%report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss = {:5.3f}, accuracy{:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\n Final Test: final accuracy = {:5.3f}'.format(final_acc))

In [11]:
def abalone_exec(epoch_count = 10, mb_size=10, report =1):
    load_abalone_dataset() # 자료 불러오기 
    init_model() # 초기화 
    train_and_test(epoch_count, mb_size, report) 
    # mb_size : mini batch size 즉 한번에 몇개씩 훈련시킬 건지 ! 사실 한꺼번에 하는 게 좋지만 현실적인 문제 때문 

In [23]:
# 모집단과 가까워지기 위해 (훈련을 잘 하기 위해 ) 자료를 섞기
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0]*0.8) // mb_size
    test_begin_idx = step_count * mb_size # 정수로 나타내주기 위해
    return step_count 

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth : mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [24]:
def run_train(x,y): 
    output, aux_nn = forward_neuralnet(x) #forward
    loss, aux_pp = forward_postproc(output, y) # loss 계산
    accuracy = eval_accuracy(output, y)
    # forward랑 순서 반대로 하면 backward
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp) 
    backprop_neuralnet(G_output, aux_nn) #loss가 줄어드는 방향으로 weight, bias 조절
    return loss, accuracy

def run_test(x,y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [26]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight)+bias
    return output,x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)
    
    # 수치 업데이트 
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [28]:
def forward_postproc(output, y):
    diff = output -y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2*diff
    g_diff_output = 1
    
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

In [29]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output-y)/y))
    return 1 - mdiff # 정확도 출력 